<a href="https://colab.research.google.com/github/Dcanbolat/hu_bby162_2024/blob/main/web_sayfas%C4%B1n%C4%B1_tarama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Gerekli kütüphaneleri yükleyin
!pip install requests beautifulsoup4

import requests
from bs4 import BeautifulSoup

# Kullanıcıdan başlangıç URL'sini ve derinliği al
start_url = input("Başlangıç URL'sini girin: ")
depth = int(input("Tarama derinliğini girin (1-3): "))

# Web sayfasını tarayan fonksiyon
def crawl(url):
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        title = soup.title.string if soup.title else 'Başlık Bulunamadı'
        links = [a['href'] for a in soup.find_all('a', href=True)]
        return title, links
    except Exception as e:
        print(f"Error crawling {url}: {e}")
        return None, []

# Tarama işlemini gerçekleştiren fonksiyon
def spider(start_url, depth):
    visited = set()
    results = []

    def crawl_recursive(url, level):
        if level > depth or url in visited:
            return
        visited.add(url)
        title, links = crawl(url)
        results.append((level, url, title))

        for link in links:
            # Linkin mutlak URL olup olmadığını kontrol edin
            if not link.startswith('http'):
                link = requests.compat.urljoin(url, link)
            crawl_recursive(link, level + 1)

    crawl_recursive(start_url, 1)
    return results

# Sonuçları kaydeden fonksiyon
def save_results(results):
    with open('results.txt', 'w') as file:
        for level, url, title in results:
            file.write(f"Seviye {level}: {url} - Başlık: {title}\n")

# Tarama işlemini başlat ve sonuçları kaydet
results = spider(start_url, depth)
save_results(results)
print("Tarama tamamlandı. Sonuçlar 'results.txt' dosyasına kaydedildi.")
